In [1]:
import overpy
import pandas as pd
import time
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point, Polygon, LineString
from geopandas import GeoDataFrame

In [2]:
# Configure osmnx to not use cache
ox.config(use_cache=False)

/var/folders/3f/3mffjgc50tdc2j3m2fqnxfxc0000gn/T/ipykernel_8456/3057731840.py:2: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=False)


In [3]:
# Function to create bounding boxes
def generate_bboxes(south, west, north, east, lat_step=1.0, lon_step=1.0):
    """Generate non-overlapping bounding boxes given a larger bounding box."""
    assert south < north, "Invalid latitude range"
    assert west < east, "Invalid longitude range"
    
    lat_ranges = [south + i*lat_step for i in range(int((north-south)/lat_step) + 1)]
    lon_ranges = [west + i*lon_step for i in range(int((east-west)/lon_step) + 1)]
    
    bboxes = [(lat_ranges[i], lon_ranges[j], lat_ranges[i+1], lon_ranges[j+1]) 
              for i in range(len(lat_ranges)-1) 
              for j in range(len(lon_ranges)-1)]
    
    return bboxes

In [4]:
# Function to create overlapping bounding boxes
def add_overlap_to_bboxes(bboxes, lat_overlap, lon_overlap, max_north, max_east, min_south, min_west):
    """Add overlap to bounding boxes without exceeding specified max and min lat and lon."""
    return [
        (max(south-lat_overlap, min_south), 
         max(west-lon_overlap, min_west),
         min(north+lat_overlap, max_north), 
         min(east+lon_overlap, max_east)) 
        for south, west, north, east in bboxes
    ]

In [5]:
# Function to fetch parking data for bbox
def fetch_parking_data_for_bbox(bbox, max_retries=3):
    """Query parking data for a given bounding box."""
    api = overpy.Overpass()
    bbox_str = ",".join(map(str, bbox))
    retries = 0
    
    while retries < max_retries:
        try:
            query = f"""
            [out:json];
            (
              node["amenity"="parking"]["access"~"public|customers|permissive|yes"]({bbox_str});
              way["amenity"="parking"]["access"~"public|customers|permissive|yes"]({bbox_str});
              relation["amenity"="parking"]["access"~"public|customers|permissive|yes"]({bbox_str});
              node["amenity"="parking"]["access"!~".*"]({bbox_str});
              way["amenity"="parking"]["access"!~".*"]({bbox_str});
              relation["amenity"="parking"]["access"!~".*"]({bbox_str});
            );
            out body;
            >;
            out skel qt;
            """
            result = api.query(query)
            return result
        except (overpy.exception.OverpassRuntimeError, Exception):
            retries += 1
            print(f"Error for bbox {bbox}. Retrying {retries}/{max_retries}...")
            time.sleep(10)

    print(f"Failed to fetch data for bbox {bbox} after {max_retries} retries.")
    return None

In [6]:
def extract_vertices_from_entity(entity):
    """Extract vertices for an entity."""
    return [(m.lon, m.lat) for m in entity.nodes if hasattr(m, "lat") and hasattr(m, "lon")]

In [7]:
def extract_way_geometry(way):
    """Extract geometry for a Way."""
    verts = extract_vertices_from_entity(way)
    
    # If the first and last vertex are the same, it's a polygon
    if verts[0] == verts[-1]:
        return Polygon(verts)
    # Otherwise, it's a line string
    else:
        return LineString(verts)

In [8]:
def extract_relation_geometry(relation):
    """Extract geometry for a Relation."""
    outer_polys = []

    for member in relation.members:
        if isinstance(member, overpy.Way) and member.role == "outer":
            verts = extract_vertices_from_entity(member)
            outer_polys.append(verts)

    # If there's only one outer boundary, it's a Polygon
    if len(outer_polys) == 1:
        return Polygon(outer_polys[0])
    # If there are multiple outer boundaries, it's a MultiPolygon
    elif len(outer_polys) > 1:
        polys = [Polygon(outer) for outer in outer_polys]
        return MultiPolygon(polys)

    return None

In [9]:

def process_result(result):
    """Process OSM result to extract parking data."""
    
    # Process nodes
    for node in result.nodes:
        add_to_data(node, node.lat, node.lon, geom=Point(node.lon, node.lat))

    # Process ways
    for way in result.ways:
        geom = extract_way_geometry(way)
        add_to_data(way, None, None, geom=geom)

    # Process relations
    for relation in result.relations:
        geom = extract_relation_geometry(relation)
        if geom:
            add_to_data(relation, None, None, geom=geom)

In [10]:
TAGS_LIST = [
    'name', 'parking', 'capacity', 'capacity:charging', 'fee',
    'supervised', 'surface', 'maxstay', 'opening_hours', 'operator',
    'website', 'zone', 'capacity:disabled', 'capacity:parent',
    'ref', 'park_ride'
]

In [11]:

def add_to_data(element, lat=None, lon=None, geom=None):
    """Extract parking data from an OSM element and append to df_data list."""
    # Check the geometry type to extract vertices accordingly
    if isinstance(geom, Point):
        vertices = [(geom.x, geom.y)]
    elif isinstance(geom, LineString):
        vertices = list(geom.coords)
    elif geom and hasattr(geom, 'exterior'):  # for Polygon
        vertices = list(geom.exterior.coords)
    else:
        vertices = None

    data_dict = {
        "ID": element.id,
        "Latitude": lat,
        "Longitude": lon,
        "Geometry": geom,
        "Vertices": vertices
    }

    for tag in TAGS_LIST:
        data_dict[tag.capitalize()] = element.tags.get(tag, 'Unknown')

    df_data.append(data_dict)

In [12]:
# Fetching Germany boundary and defining its bounding box
germany_osm = ox.geocode_to_gdf("Germany")
GERMANY_BBOX_DF = germany_osm.bounds.iloc[0]
GERMANY_BBOX = (GERMANY_BBOX_DF['miny'], GERMANY_BBOX_DF['minx'], GERMANY_BBOX_DF['maxy'], GERMANY_BBOX_DF['maxx'])

In [13]:
bboxes = generate_bboxes(*GERMANY_BBOX, lat_step=0.5, lon_step=0.5)
overlapping_bboxes = add_overlap_to_bboxes(bboxes, 0.05, 0.05, 
                                           GERMANY_BBOX[2], GERMANY_BBOX[3], 
                                           GERMANY_BBOX[0], GERMANY_BBOX[1])

In [14]:
# Fetching and processing data
all_results = []
df_data = []
for index, bbox in enumerate(overlapping_bboxes, 1):
    print(f"Fetching data for bbox {index}/{len(overlapping_bboxes)}: {bbox}")
    result = fetch_parking_data_for_bbox(bbox)
    if result:
        process_result(result)
        all_results.append(result)

Fetching data for bbox 1/270: (47.2701114, 5.8663153, 47.820111399999995, 6.4163153)
Fetching data for bbox 2/270: (47.2701114, 6.3163153, 47.820111399999995, 6.9163153)
Fetching data for bbox 3/270: (47.2701114, 6.8163153, 47.820111399999995, 7.4163153)
Fetching data for bbox 4/270: (47.2701114, 7.3163153, 47.820111399999995, 7.9163153)
Fetching data for bbox 5/270: (47.2701114, 7.8163153, 47.820111399999995, 8.4163153)
Fetching data for bbox 6/270: (47.2701114, 8.3163153, 47.820111399999995, 8.9163153)
Fetching data for bbox 7/270: (47.2701114, 8.8163153, 47.820111399999995, 9.4163153)
Fetching data for bbox 8/270: (47.2701114, 9.3163153, 47.820111399999995, 9.9163153)
Fetching data for bbox 9/270: (47.2701114, 9.8163153, 47.820111399999995, 10.4163153)
Fetching data for bbox 10/270: (47.2701114, 10.3163153, 47.820111399999995, 10.9163153)
Fetching data for bbox 11/270: (47.2701114, 10.8163153, 47.820111399999995, 11.4163153)
Fetching data for bbox 12/270: (47.2701114, 11.3163153, 47

In [15]:
# Converting to DataFrame
df = pd.DataFrame(df_data)

In [16]:
df.head(20)

,ID,Latitude,Longitude,Geometry,Vertices,Name,Parking,Capacity,Capacity:charging,Fee,...,Surface,Maxstay,Opening_hours,Operator,Website,Zone,Capacity:disabled,Capacity:parent,Ref,Park_ride
0,286220996,47.2809601,6.1273641,POINT (6.1273641 47.2809601),"[(6.1273641, 47.2809601)]",Unknown,surface,Unknown,Unknown,no,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,430719192,47.2961962,6.0310415,POINT (6.0310415 47.2961962),"[(6.0310415, 47.2961962)]",Petites Baraques,surface,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,766493016,47.6594314,6.3640626,POINT (6.3640626 47.6594314),"[(6.3640626, 47.6594314)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,1091480538,47.3214487,6.3639424,POINT (6.3639424 47.3214487),"[(6.3639424, 47.3214487)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,1091480559,47.3151728,6.3679156,POINT (6.3679156 47.3151728),"[(6.3679156, 47.3151728)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
5,1171660850,47.3273776,6.0121072,POINT (6.0121072 47.3273776),"[(6.0121072, 47.3273776)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
6,1171660856,47.3258578,6.0144885,POINT (6.0144885 47.3258578),"[(6.0144885, 47.3258578)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
7,1171660882,47.3269559,6.0077722,POINT (6.0077722 47.3269559),"[(6.0077722, 47.3269559)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
8,1171660884,47.3293047,6.0122461,POINT (6.0122461 47.3293047),"[(6.0122461, 47.3293047)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
9,1171660890,47.3303519,6.0188444,POINT (6.0188444 47.3303519),"[(6.0188444, 47.3303519)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [17]:
len(df)

6799364

In [18]:
# Remove duplicates 
df_cleaned = df.drop_duplicates(subset='ID', keep='first')
print(len(df_cleaned))

4831949


In [19]:
# Convert the dataframe to a GeoDataFrame with appropriate geometry
gdf = gpd.GeoDataFrame(df_cleaned, geometry='Geometry', crs="EPSG:4326")

In [20]:
parking_in_germany = gpd.sjoin(gdf, germany_osm, how="inner", predicate="within")

# Drop the unnecessary index and columns introduced from germany_osm after join
original_columns = gdf.columns
parking_in_germany = parking_in_germany[original_columns]

In [21]:
len(parking_in_germany)

3689321

In [22]:
parking_in_germany.head(20)

,ID,Latitude,Longitude,Geometry,Vertices,Name,Parking,Capacity,Capacity:charging,Fee,...,Surface,Maxstay,Opening_hours,Operator,Website,Zone,Capacity:disabled,Capacity:parent,Ref,Park_ride
65248,9323857,47.7148176,7.6537833,POINT (7.65378 47.71482),"[(7.6537833, 47.7148176)]",Unknown,surface,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65249,21307663,47.7893012,7.5377884,POINT (7.53779 47.78930),"[(7.5377884, 47.7893012)]",Unknown,lane,Unknown,Unknown,no,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65250,27124796,47.6300774,7.5839347,POINT (7.58393 47.63008),"[(7.5839347, 47.6300774)]",Unknown,surface,Unknown,Unknown,no,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65251,27358093,47.5905453,7.5929529,POINT (7.59295 47.59055),"[(7.5929529, 47.5905453)]",Unknown,multi-storey,Unknown,Unknown,no,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65252,35108095,47.5806549,7.7864517,POINT (7.78645 47.58065),"[(7.7864517, 47.5806549)]",Unknown,surface,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65253,35262861,47.7317643,7.5582767,POINT (7.55828 47.73176),"[(7.5582767, 47.7317643)]",Parkplatz an der Kirche,surface,25,Unknown,no,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65254,35334594,47.7280664,7.5936679,POINT (7.59367 47.72807),"[(7.5936679, 47.7280664)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65255,35360209,47.7161753,7.5586050,POINT (7.55861 47.71618),"[(7.558605, 47.7161753)]",Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65256,36691854,47.6578978,7.6042919,POINT (7.60429 47.65790),"[(7.6042919, 47.6578978)]",Unknown,surface,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
65257,36694168,47.6125462,7.5921697,POINT (7.59217 47.61255),"[(7.5921697, 47.6125462)]",Unknown,lane,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [23]:
# Convert the GeoDataFrame to the desired CRS and add areas in m2 to the polygons
parking_in_germany = parking_in_germany.to_crs("EPSG:25832")
parking_in_germany["Area_m2"] = parking_in_germany["Geometry"].area

In [24]:
# Save as CSV
csv_path = "~/Desktop/Masterarbeit/data/osm_parking_data_fetching.csv"
parking_in_germany.to_csv(csv_path, index=False)

In [25]:
# Convert Latitude and Longitude to float
parking_in_germany['Latitude'] = parking_in_germany['Latitude'].astype(float)
parking_in_germany['Longitude'] = parking_in_germany['Longitude'].astype(float)

In [26]:
# Convert Vertices to string
parking_in_germany['Vertices'] = parking_in_germany['Longitude'].astype(str)

In [27]:
# Select only necessary columns
subset_gdf = parking_in_germany[['ID', 'Latitude', 'Longitude', 'Geometry', 'Vertices']]

In [29]:
# Save to GPKG
file_path = "~/Desktop/Masterarbeit/osm_parking_areas_in_germany.gpkg"
subset_gdf.to_file(file_path, driver='GPKG')